In [2]:
import csv   

In [3]:
def downloadAllData(reader):
    L = []
    for row in reader:
        L.append(row)
    return L

In [4]:
def predictMin(data, startIndex, windowsSize):
    totalNumber = len(data)
    minSet = []
    for j in range(startIndex, startIndex + windowsSize):   
        minSet.append(float(data[j][1]))
    result = sorted(minSet)
    return result

In [5]:
def getPpercent(percent, list):
    total = len(list)
    index = float(total*percent)
    if float(index) - float(int(index)) == 0:
        index = int(index)
        return float((list[index]+list[index-1])/2)
    index = int(index)
    return float(list[index-1]) 

In [6]:
def predictValue(list, windowSize, percent):
    result = []
    # The last index is unuse
    for i in range(1,len(list)-windowSize):
        tempList = predictMin(list, i, windowSize)
        pValue = getPpercent(percent, tempList)
        result.append(pValue)
    return result

In [7]:
def getTrueMin(data, startblock):
    result=[]
    # start point
    for i in range(startblock, len(data)):
        result.append(float(data[i][1]))
    return result

In [8]:
def computeOnChainPercentFixRange(predict, startIndex, endIndex, trueData):
    possiblityOnchain = 0
    integerRange = endIndex - startIndex
    for i in range(startIndex, endIndex):
        if float(predict[i]) > float(trueData[i]):
            possiblityOnchain +=1
    return float(possiblityOnchain / integerRange)

In [9]:
def getLargeBias(winSize, predict, trueData):
    upperBound = len(predict) - winSize
    largeBias = computeOnChainPercentFixRange(predict, 0, winSize, trueData)
    smallNumber = 0
    for i in range(1, upperBound):
        temp = computeOnChainPercentFixRange(predict, i, winSize+i, trueData)
        if temp < largeBias:
            largeBias = temp
            smallNumber = i
    return largeBias, smallNumber

In [10]:
#GasStation-Express： Use 200 blocks
csvFile = open("/Users/chihyunchuang/Downloads/new_block_p2.5.csv")
reader = csv.reader(csvFile)
data = downloadAllData(reader)

GSP50 = predictValue(data, 200, 0.5)
GSP75 = predictValue(data, 200, 0.75)
GSP84 = predictValue(data, 200, 0.8413)
GSP95 = predictValue(data, 200, 0.95)
trueMin = getTrueMin(data,201)

In [11]:
#ETHGasStation Case： Use 100 blocks 
csvFile = open("/Users/chihyunchuang/Downloads/new_block_p2.5.csv")
reader = csv.reader(csvFile)
data = downloadAllData(reader)

expectedGasStationP50 = predictValue(data, 100, 0.5)
expectedGasStationP75 = predictValue(data, 100, 0.75)
expectedGasStationP84 = predictValue(data, 100, 0.8413)
expectedGasStationP95 = predictValue(data, 100, 0.95)

# Remove first 100 blocks
expectedGasStationP50 = expectedGasStationP50[100: len(expectedGasStationP50)]
expectedGasStationP75 = expectedGasStationP75[100: len(expectedGasStationP75)]
expectedGasStationP84 = expectedGasStationP84[100: len(expectedGasStationP84)]
expectedGasStationP95 = expectedGasStationP95[100: len(expectedGasStationP95)]

In [12]:
csvFile = open("/Users/chihyunchuang/Downloads/new1-136312.csv")
reader2 = csv.reader(csvFile)
datGP = downloadAllData(reader2)
datGP = datGP[1:]

In [13]:
with open("/Users/chihyunchuang/Downloads/newDataForNewBlock.csv", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['block_number','true_min', 'GP-50', 'GP-75', 'GP-84.13', 'GP-95',   'Geth-50', 'Geth-75', 'Geth-84.13', 'Geth-95', 'GasStation-50', 'GasStation-75', 'GasStation-84.13', 'GasStation-95' ])

    for i in range(1,len(datGP)-1):
        writer.writerow([str(data[i+201][0]), trueMin[i], float(datGP[i][1]), float(datGP[i][1])+0.675*float(datGP[i][2]), float(datGP[i][1])+float(datGP[i][2]), float(datGP[i][1])+1.645*float(datGP[i][2])  ,  expectedGasStationP50[i], expectedGasStationP75[i], expectedGasStationP84[i], expectedGasStationP95[i], GSP50[i], GSP75[i],GSP84[i], GSP95[i]  ])